## Train an agent to play tic tac toe

### Strategies
1. Play at random
2. Ideal Player
3. Imitation learning


In [27]:
import itertools
import random
import time

In [58]:
DEBUG = False
def log(s):
    if DEBUG: print(s)


In [73]:
COMPUTER = False
HUMAN = True

class Player():
    HUMAN = 0
    RANDOM = 1
    EXPERT = 2
    STUDENT = 3
    
    def __init__(self, player, symbol):
        if (player == Player.HUMAN
                or player == Player.RANDOM
                or player == Player.EXPERT
                or player == Player.STUDENT):
            self.ptype = player
            self.name = Player.get_player_str(player)
            self.symbol = symbol
        else:
            raise ValueError(f'{player} is not a valid Player identifier.')
            
    def __str__(self):
        return f'ptype: {self.ptype}, name: {self.name}, symb: {self.symbol}'

    @staticmethod
    def get_player_str(player):
        if player == Player.HUMAN:
            return 'HUMAN'
        elif player == Player.RANDOM:
            return 'COMP-RANDOM'
        elif player == Player.EXPERT:
            return 'COMP-EXPERT'
        elif player == Player.STUDENT:
            return 'COMP-STUDENT'


class Entry():
    Empty = '-'
    X = 'X'
    O = 'O'
    

In [103]:


class TicTacToe():
    """
    define the game class
    COMPUTER always plays an 'O'
    HUMAN always plays a 'X'
    """
    
    MNMX_MOVES = 0
    TURNS = 0
    
    def __init__(self, player_1, player_2):
        """
        turn = False -> computer's turn, True-> human turn
        """
        if(player_1 == player_2) :
            print("ERROR: same players not supported right now")
            assert(player_1 != player_2)
            return 
        self.state = ['-'] * 9
        self.turn = random.choice([player_1, player_2])
        self.game_ended = False
        self.winner = '-'
        self.player_1 = player_1
        self.player_2 = player_2
        self.prev_state = self.state
        self.last_played = -1
        
    def reset(self):
        self.state = ['-'] * 9
        self.turn = random.choice([self.player_1, self.player_2])
        self.game_ended = False
        self.winner = '-'
        self.prev_state = self.state
        self.cur_state = self.state
        self.last_played = -1
        
    def _switch_turn(self):
        self.turn = self._get_opponent(self.turn)
            
    def _get_opponent(self, player):
        if player == self.player_1:
            return self.player_2
        else:
            return self.player_1
     
    def __str__(self):
        x = self.pretty_state(self.state)
        return (f'board state: \n{x}\n' +
                f'player turn: {self.turn}\n' +
                f'game ended: {self.game_ended}\n' +
                f'winner: {self.winner}\n')
    
    def pretty_state(self, state):
        x = str(state[0:3]) + '\n' + \
            str(state[3:6]) + '\n' + \
            str(state[6:9])
        return x
    
    def play(self):
        self.prev_state = self.state.copy()
        self.TURNS += 1
        log(f'TURN no: {self.TURNS}')
        log('play a turn')
        log(f'turn: {self.turn}')
        if self.game_ended:
            log('Game Over')
            log(self)
            return
        avail_positions = []
        for i, x in enumerate(self.state):
            if x == Entry.Empty: avail_positions.append(i)
        if len(avail_positions) == 0:
            self.game_ended = True
            self.winner = 'DRAW'
            log('board is full')
            return
        if self.turn.ptype == Player.RANDOM:
            log(f'{self.turn.name} to play')
            log(f'available positions: {avail_positions}')
            play_id = random.choice(avail_positions) 
            log(play_id)
            self.state[play_id] = self.turn.symbol
            self.last_played = play_id
        elif self.turn.ptype == Player.EXPERT:
            play_id = self.play_pro(self.turn)
            self.state[play_id] = self.turn.symbol
            self.last_played = play_id
        elif self.turn == Player.HUMAN:
            print('HUMAN to play')
            print(f'available positions: {avail_positions}')
            self.user_input_prompt()
            valid_input = False
            while not valid_input:
                inp = input('where do you wanna play [0-9]?')
                if str.isdigit(inp): valid_input = True
                if valid_input:
                    pos = int(inp)
                    if pos not in avail_positions:
                        valid_input = False
                if not valid_input:
                    print('invalid input')
                    print(f'please enter a number from the list: {avail_positions}')
            # got a valid position to play
            self.state[pos] = self.turn.symbol
        
        self.evaluate()
#         self.turn = not self.turn
        self._switch_turn()
        log(self)
#         time.sleep(1)
#         print(f'winner: {self.winner}')
        
    def play_pro(self, player):
        """
        returns the best move by
            play as an expert(pro) using minimax
        `player` object which is playing as pro
        """
        state_copy = self.state.copy()
        self.MNMX_MOVES = 0
        best_move, best_score = self._minimax(state_copy, player)
        log(f'minimax moves taken: {self.MNMX_MOVES}')
        return best_move
        
    def _evaluate(self, state):
        """
        evaluate state, returns game_ended
        """
        rows = [self.state[k:k+3] for k in range(0, 9, 3)]
        cols = [[self.state[k], self.state[k+3], self.state[k+6]]
                 for k in range(0, 3, 1)]
        diags = [[self.state[0], self.state[4], self.state[8]],
                 [self.state[2], self.state[4], self.state[6]]]
        arrs = [rows, cols, diags]
        for arr in itertools.chain(*arrs):
            if (arr[0] != Entry.Empty
                    and arr[0] == arr[1]
                    and arr[0] == arr[2]):
                return True
        return False
        
    def _minimax(self, state, player):
        """
        finds the best move that `player` can play with the 
            given board `state`
        """
        self.MNMX_MOVES += 1
#         print(f'enter mnmx with state:\n{self.pretty_state(state)}')
        empty_pos = self.get_available_pos(state)
        if len(empty_pos) == 0:
            log('no moves available. exiting!')
            log(f'player: {Player.get_player_str(player)}')
        new_state = self.state
        best_score = -100
        best_move = -1
        for pos in empty_pos:
#             print(f'make move: {pos}')
#             if player == COMPUTER: new_state[pos] = Entry.O
#             else: new_state[pos] = Entry.X
            new_state[pos] = player.symbol
            
            if self._evaluate(new_state): # played the winning move
#                 print('winning minimax move')
#                 print(f'player: {player}, state:\n{state}')
#                 return pos, 10
                cur_score = 10
                # try early stopping
                best_score = cur_score
                best_move = pos
                new_state[pos] = Entry.Empty
                break
                # early stop ends
            else:
                cur_score = -100
                if len(empty_pos) == 1: # draw state, last move
                    cur_score = 0
                else:
                    # play more
                    opp = self._get_opponent(player)
                    _, opp_score = self._minimax(new_state, opp)
                    cur_score = -opp_score
            if cur_score > best_score:
                best_score = cur_score
                best_move = pos
            # reset state
            new_state[pos] = Entry.Empty
#             print(f'UNDO move: {pos}')
#         print(f'player: {player}, best_move = {pos}, best_score = {best_score}')
#         print(f'exit mnmx with state:\n{self.pretty_state(state)}')
        return best_move, best_score
        
    def evaluate(self):
        """
        evaluate if there is a winner
        if game ended, update `game_ended` and `winner`
        """
        win = False
        # check rows
        rows = [self.state[k:k+3] for k in range(0, 9, 3)]
        cols = [[self.state[k], self.state[k+3], self.state[k+6]]
                 for k in range(0, 3, 1)]
        diags = [[self.state[0], self.state[4], self.state[8]],
                 [self.state[2], self.state[4], self.state[6]]]
        arrs = [rows, cols, diags]
        for arr in itertools.chain(*arrs):
            if (arr[0] != Entry.Empty
                    and arr[0] == arr[1]
                    and arr[0] == arr[2]):
                win = True
                log(f'winning row: {arr}')
                break
        if win:
            log('we have a winner')
            self.winner = self.turn
#             if self.turn: self.winner = "HUMAN"
#             else: self.winner = "COMPUTER"
            self.game_ended = True
            
    def get_available_pos(self, state):
        avail_positions = []
        for i, x in enumerate(state):
            if x == Entry.Empty: avail_positions.append(i)
        return avail_positions
    
#     def get_state(self):
#         state = 0
#         for i in range(9):
#             s = self.state[i]
#             val = 0
#             if s == Entry.X:
#                 val = 0x3
#             elif s == Entry.O:
#                 val = 0x2
#             state |= val << (i*2)
#         return state
        
    def user_input_prompt(self):
        """
        shows prompt human user to get position to play
        """
        prompt = ''
        for i, x in enumerate(self.state):
            prompt += f'[{i}| {x}]'
            if (i+1) % 3 == 0: prompt += '\n'
        
        print(f'board state: \n{prompt}\n')
        
    


In [131]:
import json
import time

memory = []
mem_set = set()

def simulate_rand_v_expert(trials):
    noob = Player(Player.RANDOM, Entry.O)
    pro  = Player(Player.EXPERT, Entry.X)
    game = TicTacToe(noob, pro)
    log(game)
    win_n, win_pro, win_d, win_e = 0, 0, 0, 0
    t_start = time.time()
    for i in range(trials):
        while not game.game_ended:
            game.play()
            if game.turn == noob: # pro played last turn
                memory.append({
                    "prev_state": game.prev_state,
                    "last_played": game.last_played
                })
                temp_a = game.prev_state.copy()
                temp_a.append(game.last_played)
                mem_set.add(tuple(temp_a))
        if game.winner == noob:
            win_n += 1
        elif game.winner == pro:
            win_pro += 1
        elif game.winner == 'DRAW':
            win_d += 1
        elif game.winner == '-':
            win_e += 1
        else:
            print('something else')
        game.reset()
        if i%50 == 0:
            t_end = time.time()
            print(f'trial: {i}, total time taken: {round(t_end-t_start, 2)}s')

    print('simulate done')
    return win_n, win_pro, win_d, win_e
    
win_n, win_pro, win_d, win_e = simulate_rand_v_expert(1000)
print(f'noob win: {win_n}, pro wins: {win_pro}')
print(f'draws: {win_d}, e wins: {win_e}')
# for mem in memory:
#     print(json.dumps(mem))
print(f'mem size: {len(memory)}, mem_set size: {len(mem_set)}')


trial: 0, total time taken: 1.39s
trial: 50, total time taken: 38.3s
trial: 100, total time taken: 80.4s
trial: 150, total time taken: 112.66s
trial: 200, total time taken: 158.97s
trial: 250, total time taken: 189.76s
trial: 300, total time taken: 233.36s
trial: 350, total time taken: 271.12s
trial: 400, total time taken: 308.71s
trial: 450, total time taken: 341.19s
trial: 500, total time taken: 376.8s
trial: 550, total time taken: 414.96s
trial: 600, total time taken: 453.91s
trial: 650, total time taken: 491.85s
trial: 700, total time taken: 541.4s
trial: 750, total time taken: 581.72s
trial: 800, total time taken: 622.22s
trial: 850, total time taken: 664.88s
trial: 900, total time taken: 703.02s
trial: 950, total time taken: 740.91s
simulate done
noob win: 0, pro wins: 891
draws: 109, e wins: 0
mem size: 3315, mem_set size: 430


In [137]:
# write `mem_set` to file
with open('ttt-mem.txt', 'w') as f:
    f.write(str(mem_set))

print('write to file')


write to file


In [ ]:

# Next Steps:

# train a NN on these 430 examples
# see / measure how it performs


# transform the data set
# and train again

In [38]:
def play_new_game(game):
    print(f'old game state: {game}')
    game.reset()
    while not game.game_ended:
        game.play()
        
    print('done.')

In [119]:
xx = ['x', 'o']
zz = xx + 2
yy = set()
yy.add(tuple(zz))
yy

TypeError: can only concatenate list (not "int") to list

In [50]:
obj = Test()
obj.check()
print(obj)
for i in range(2):
    obj.func()
print(obj)
obj.check()

self.Y: 70
x: 20, y: 70
x: 22, y: 72
self.Y: 72


In [ ]:
3**3

In [51]:
x = random.choice([Player.RANDOM, Player.EXPERT])
print(Player.get_player_str(x))

COMP-EXPERT
